<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2022/blob/main/notebooks/7_doubly_robust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pakiety i dane

Ładujemy pakiety

In [1]:
library(tidyverse) 

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Pobieramy dane z zajęć

In [3]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")

In [4]:
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE)
head(dane)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA


Zrobimy bardzo uproszczone podejście, że działamy wyłącznie na próbie losowej i w niej identyfikujemy oferty z CBOP.

In [5]:
data_for_models <- dane %>%
  filter(!is.na(id_popyt)) %>%
  mutate(waga_final = waga*wolne_miejsca,
         waga_final_scaled = waga_final/mean(waga_final),
         R = !is.na(id_cbop),
         zawod_kod2 = as.factor(zawod_kod2),
         woj = as.factor(woj))
head(data_for_models)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop,waga_final,waga_final_scaled,R
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<fct>,<fct>,<int>,<int>,<lgl>,<lgl>,<int>,<int>,<int>,<dbl>,<lgl>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA,2,0.06778256,FALSE
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA,7,0.23723895,FALSE
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA,36,1.22008604,FALSE
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA,7,0.23723895,FALSE
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA,6,0.20334767,FALSE
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA,1,0.03389128,FALSE


Regresja logistyczna wyjaśniająca zmienną `R` oznaczającą przynależność do CBOP.

In [6]:
model_ps <- glm(formula = R ~ sek + klasa_pr  + woj + sekc_pkd+  zawod_kod2,
                data = data_for_models,
                family = binomial(),
                weights = waga_final_scaled)

summary(model_ps)

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”



Call:
glm(formula = R ~ sek + klasa_pr + woj + sekc_pkd + zawod_kod2, 
    family = binomial(), data = data_for_models, weights = waga_final_scaled)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.3653  -0.2123  -0.1264  -0.0679  10.6440  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -3.90911    0.88616  -4.411 1.03e-05 ***
sek         -0.41982    0.25401  -1.653 0.098371 .  
klasa_prM   -1.39778    0.14373  -9.725  < 2e-16 ***
klasa_prS   -1.78411    0.18561  -9.612  < 2e-16 ***
woj4         0.33602    0.35765   0.940 0.347463    
woj6        -0.53881    0.37032  -1.455 0.145675    
woj8        -0.38494    0.54631  -0.705 0.481046    
woj10       -0.70847    0.38288  -1.850 0.064261 .  
woj12       -0.32880    0.30790  -1.068 0.285577    
woj14        0.16457    0.23835   0.690 0.489899    
woj16        0.47789    0.44627   1.071 0.284235    
woj18        2.58223    0.26581   9.714  < 2e-16 ***
woj20        0.61657    0.43666 

Następnie budujemy model wyjaśniajacy kolumnę `jedna_zmiana`.

In [7]:
model_y <- glm(formula = jedna_zmiana ~ sek + klasa_pr  + woj + sekc_pkd+  zawod_kod2,
               data = data_for_models,
               subset = R == TRUE,
               family = binomial(),
               weights = waga_final_scaled)
summary(model_y)

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”



Call:
glm(formula = jedna_zmiana ~ sek + klasa_pr + woj + sekc_pkd + 
    zawod_kod2, family = binomial(), data = data_for_models, 
    weights = waga_final_scaled, subset = R == TRUE)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.7180  -0.4479  -0.1159   0.3953   3.3222  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.22406    2.43927   0.092 0.926812    
sek         -0.04156    0.78837  -0.053 0.957962    
klasa_prM    5.48045    0.94231   5.816 6.03e-09 ***
klasa_prS    0.88866    0.59022   1.506 0.132158    
woj4        -0.95889    0.85013  -1.128 0.259350    
woj6        -0.59377    1.35753  -0.437 0.661828    
woj8        -0.52984    1.26894  -0.418 0.676279    
woj10       -3.44924    1.33994  -2.574 0.010048 *  
woj12        0.54816    0.85414   0.642 0.521022    
woj14       -2.35971    0.67360  -3.503 0.000460 ***
woj16        0.89289    1.08538   0.823 0.410707    
woj18       -0.12864    0.77620  -0.166 0.868365 

Dodajemy $\hat{\rho}$ oraz $\hat{m}$ zgodnie z wzorami ze slajdów.

In [9]:
data_for_models <- data_for_models %>%
  mutate(rho_hat = predict(model_ps, data_for_models, type = "response"),
         w_hat = 1/ rho_hat,
         m_hat = predict(model_y, data_for_models, type = "response"))
head(data_for_models)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop,waga_final,waga_final_scaled,R,rho_hat,w_hat,m_hat
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<fct>,<fct>,<int>,<int>,<lgl>,<lgl>,<int>,<int>,<int>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA,2,0.06778256,FALSE,0.004251277,235.22345,0.50319928
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA,7,0.23723895,FALSE,0.010938389,91.42114,0.36449427
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA,36,1.22008604,FALSE,0.012663713,78.96578,0.56199453
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA,7,0.23723895,FALSE,0.062403515,16.02474,0.19768744
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA,6,0.20334767,FALSE,0.062403515,16.02474,0.19768744
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA,1,0.03389128,FALSE,0.047079195,21.24080,0.09696319


Wyznaczamy estymator DR oraz dla porównania: naiwny (ważona średnia arytmetyczna), propensity score estimator (ważony $1/\hat{\rho}$) oraz masową imputację ($\hat{m}$).

In [11]:
dr_part1 <- data_for_models %>%
  filter(R == TRUE) %>%
  summarise(theta_hat = weighted.mean(jedna_zmiana - m_hat, w_hat)) %>%
  pull(theta_hat)

dr_part2 <- data_for_models %>%
  filter(R == FALSE) %>%
  summarise(theta_hat = weighted.mean(m_hat, waga_final_scaled)) %>%
  pull(theta_hat)

naive <- data_for_models %>%
  filter(R == TRUE) %>%
  summarise(theta_hat = weighted.mean(jedna_zmiana, waga_final_scaled)) %>%
  pull(theta_hat)

ps <- data_for_models %>%
  filter(R == TRUE) %>%
  summarise(theta_hat = weighted.mean(jedna_zmiana, waga_final_scaled*w_hat)) %>%
  pull(theta_hat)

results <- c(dr_part1 = dr_part1, dr_part2 = dr_part2, theta_dr = dr_part1+dr_part2, naive = naive, ps = ps, mi = dr_part2)

print(results)

 dr_part1  dr_part2  theta_dr     naive        ps        mi 
0.1057680 0.6502704 0.7560384 0.5499774 0.6179120 0.6502704 
